## Speeding up the god forsakenly slow d_G_exchange function.

### Testing Environment

To Standarize the testing of all of these methods, the inputs given to the d_G_exchange function by the first guide generated by the current tool using `GFP.fasta` as an input and the `E_coli_Genome.fasta` and `GFP.fasta` as the output. This should serve as a good use case for meta genomic processing. I'm also making an assumption that if new tools are faster it is worth importing them. 

### Methods to Test out and implement:

* Parallelization 
* Iteration 
* Numpy/Pandas to perform tabular functions
* Map 

These are some optimizations we can make towards the end:

* Numba
* Pypy
* Cython



In [9]:
import pickle
import numpy as np
import scipy.io
from itertools import repeat
import numba as nb
from numba.decorators import jit, autojit, njit

__CODE SETUP__

In [10]:
#Variable Files: 
f = open('Cas_Var.pckl', 'rb')
genome_dictionary = pickle.load(f)
f.close()

mat = scipy.io.loadmat('InvitroModel.mat')
weights = mat['w1']

#Guide
guide = 'CAAACTCAAGAAGGACCATG'

#Functions
def QuickCalc_Exchange_Energy(crRNA,TargetSeq):
    nt_pos={'A':0,'T':1,'C':2,'G':3,'a':0,'t':1,'c':2,'g':3}
    dG=0
    RNA=''
    DNA=''
    nt_mismatch_in_first8=0
    for i in range(0,len(crRNA)):
        pos=20-i
        w1=weights[pos]
        if nt_pos[crRNA[i]]==nt_pos[TargetSeq[i]]:
            dG1=0
        else:
            # using a bioinformatics search approach to find sequences with up to x mismatches
            dG1=2.3 # kcal/mol
            if pos<=8:
                nt_mismatch_in_first8=nt_mismatch_in_first8+1
        dG+=w1*dG1
    return float(dG)

PAM_energy={'GGA':-9.8,'GGT':-10,'GGC':-10,'GGG':-9.9,'CGG':-8.1,'TGG':-7.8,'AGG':-8.1,'AGC':-8.1,'AGT':-8.1,'AGA':-7.9,'GCT':-7.1,'GCG':-6.9,'ATT':-7.2,'ATC':-6.4,'TTT':-7.6,'TTG':-6.8,'GTA':-7.4,'GTT':-7.9,'GTG':-7.7,'AAT':-7,'AAG':-7,'TAT':-7.2,'TAG':-7.2,'GAA':-7.2,'GAT':-7.3,'GAC':-7.2,'GAG':-7.3}

def returnAllPAMs():

    for (PAMpart,energies) in sorted(list(PAM_energy.items()), key = lambda x: x[1]):  #PAMpart will be 'GGT'
        for nt in ('A','G','C','T'):        #nt + PAMpart will be all possible 'NGGT'
            yield nt + PAMpart
            


__Current Code__

In [430]:
%%timeit 
for (source, targets) in genome_dictionary.items():
    for fullPAM in returnAllPAMs():
        dG_PAM = 0
        dG_supercoiling= 0
        for (target_sequence, targetPosition) in genome_dictionary[source][fullPAM]:
            dG_exchange = QuickCalc_Exchange_Energy(guide, target_sequence)

2min 41s ± 15.7 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


__Current Loop__

In [431]:
%%timeit 
for (source, targets) in genome_dictionary.items():
    for fullPAM in returnAllPAMs():
        dG_PAM = 0
        dG_supercoiling= 0
        for (target_sequence, targetPosition) in genome_dictionary[source][fullPAM]:
            dG_exchange = 0

130 ms ± 2.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


__New Loop__

In [432]:
%%timeit 
for (source, targets) in genome_dictionary.items():
    dG_dictionary = {}
    for fullPAM in returnAllPAMs():
        dG_PAM = 0
        dG_supercoiling= 0
        for (target_sequence, targetPosition) in genome_dictionary[source][fullPAM]:
            if target_sequence in dG_dictionary:
                dG_exchange = dG_dictionary[target_sequence]
            else:
                dG_exchange = 0
                dG_dictionary[target_sequence] = dG_exchange

1.31 s ± 128 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


__New Loop with Reverse Dictionary__

__Loop Comparision__

In [438]:
count = 0
repeated = 0

for (source, targets) in genome_dictionary.items():
    dG_dictionary = {}
    for fullPAM in returnAllPAMs():
        dG_PAM = 0
        dG_supercoiling= 0
        for (target_sequence, targetPosition) in genome_dictionary[source][fullPAM]:
            try:
                target_sequence in dG_dictionary
                dG_exchange = dG_dictionary[target_sequence]
                repeated +=1
            except:
                dG_exchange = 0
                dG_dictionary[target_sequence] = dG_exchange
            count +=1

print("Number of times guides were repeated out of total: ")
                
print(repeated , "/" , count)

print("The final output should be", repeated/count*100, "% faster")

Number of times guides were repeated out of total: 
88915 / 3888558
The final output should be 2.2865802696012247 % faster


In [439]:
%%timeit 
for (source, targets) in genome_dictionary.items():
    dG_dictionary = {}
    for fullPAM in returnAllPAMs():
        dG_PAM = 0
        dG_supercoiling= 0
        for (target_sequence, targetPosition) in genome_dictionary[source][fullPAM]:
            try:
                dG_exchange = dG_dictionary[target_sequence]
            except:
                dG_exchange = QuickCalc_Exchange_Energy(guide, target_sequence)
                dG_dictionary[target_sequence] = dG_exchange

3min 2s ± 7.88 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


__Current dG_exchange Function and Loop__

In [87]:
%%timeit -r 10 -n 5
dGs = {} # I consider this equalvalent to a list  because it doesn't matter 
         # from a time percpective since it will later interate over all 
         # values anyway

for (target_sequence, targetPosition) in genome_dictionary["Run_Genome_Plus_RC"]["AGGT"]:
    dG_exchange = QuickCalc_Exchange_Energy(guide, target_sequence)
    dGs[target_sequence] = {'dG_exchange': dG_exchange}

1 s ± 16.6 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


__Mapping the dG_exchange loop__

In [88]:
%%timeit -r 10 -n 5
dG_exchanges = map(QuickCalc_Exchange_Energy, repeat(guide), (x[0] for x in genome_dictionary["Run_Genome_Plus_RC"]["AGGT"]))
list(dG_exchanges)

1e+03 ms ± 9.19 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


__List Comphrension on the dG_exchange loop__

In [86]:
%%timeit -r 10 -n 5
dG_exchanges = [QuickCalc_Exchange_Energy(guide, off_target[0]) for off_target in genome_dictionary["Run_Genome_Plus_RC"]["AGGT"]]
list(dG_exchanges)

1.07 s ± 43.7 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


__New dG_exchange__

In [385]:
# New Function
def QuickCalc_Exchange_Energy_New(crRNA,TargetSeq):
    dG=0
    for i,nt in enumerate(crRNA):
        pos=20-i
        w1=weights[pos]
        if nt == TargetSeq[i]:
            continue
        else:
            dG+= w1*2.3
    return float(dG)

In [395]:
weights[19]

array([1.58135744e-05])

In [362]:
%%timeit -r 10 -n 5
dGs = {}

for (target_sequence, targetPosition) in genome_dictionary["Run_Genome_Plus_RC"]["AGGT"]:
    dG_exchange = QuickCalc_Exchange_Energy_New(guide, target_sequence)
    dGs[target_sequence] = {'dG_exchange': dG_exchange}

780 ms ± 23.5 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


__Iterator Test__

In [92]:
# New Iterator Function
def QuickCalc_Exchange_Energy_Iterator(crRNA,TargetSeq):
    dG=0
    for i,nt in enumerate(crRNA):
        pos = 20 - i
        w1=weights[pos]
        if nt == TargetSeq[i]:
            dG+= w1*0
        else:
            dG+= w1*2.3
    yield float(dG)

In [95]:
%%timeit -r 10 -n 5
dGs = {}

for (target_sequence, targetPosition) in genome_dictionary["Run_Genome_Plus_RC"]["AGGT"]:
    dG_exchange = next(QuickCalc_Exchange_Energy_Iterator(guide, target_sequence))
    dGs[target_sequence] = {'dG_exchange': dG_exchange}

1.08 s ± 40 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


As you can see iterators would work well if we had really large datasets, but we have a small dataset repeated mutliple times.

__Numpy Approach__

I had one but it failed

__Formatting the Weight Array__

The current weight array is formatted strangely, and so if you are able to reformat it initially, you should be able to speed up the code. The reformat takes about 5us so it is not a problem

In [31]:
new_weights = [weight[0]*2.3 for weight in weights][0:21]

def QuickCalc_Exchange_Energy_NewWeight(crRNA,TargetSeq):
    dG=0
    for i,nt in enumerate(crRNA):
        pos=20-i
        if nt == TargetSeq[i]:
            continue
        else:
            dG+=new_weights[pos]
    return float(dG)

In [32]:
%%timeit -r 10 -n 5

for (target_sequence, targetPosition) in genome_dictionary["Run_Genome_Plus_RC"]["AGGT"]:
    dG_exchange = QuickCalc_Exchange_Energy_NewWeight(guide, target_sequence)

77.6 ms ± 1.96 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


This is an improvement of about 12x. This should allow us to get significantly faster on the code. However, so far, no modifications have been made to actual technique of calculating dG_exchange. It takes about ~50 ms for the loop to run and apparently about ~30 ms for the weights array to be called. Both of these should be upgradable using vectorization

In [441]:
%%timeit

for (source, targets) in genome_dictionary.items():
    for fullPAM in returnAllPAMs():
        dG_PAM = 0
        dG_supercoiling= 0
        for (target_sequence, targetPosition) in genome_dictionary[source][fullPAM]:
            dG_exchange = QuickCalc_Exchange_Energy_NewWeight(guide, target_sequence)

14 s ± 228 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


__Numba this shit__

In [664]:
def QuickCalc_Exchange_Energy_PreNumba(crRNA,TargetSeq, new_weights):
    dG=0
    for i in range(len(crRNA)):
        pos=20-i
        if crRNA[i] == TargetSeq[i]:
            continue
        else:
            dG+=new_weights[pos]
    return float(dG)

In [665]:
%%timeit

for (target_sequence, targetPosition) in genome_dictionary[source][fullPAM]:
            dG_exchange = QuickCalc_Exchange_Energy_PreNumba(guide, target_sequence, new_weights)

115 ms ± 1.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
nt_pos={'A':0,'T':1,'C':2,'G':3}
num_guide = np.array([nt_pos[nt] for nt in list(guide)])


@jit()
def QuickCalc_Exchange_Energy_Numba(crRNA,TargetSeq, np_weights):
    
    dG=0
    for i in range(len(crRNA)):
        pos=20-i
        if  crRNA[i] == TargetSeq[i]:
            continue
        else:
            dG+=np_weights[pos]
    return float(dG)



In [849]:
%%timeit
dG_exchange = QuickCalc_Exchange_Energy_Numba(num_guide, num_target, np_weights)

508 ns ± 7.31 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [760]:
%%timeit
QuickCalc_Exchange_Energy_NewWeight(guide, target_sequence)

3.03 µs ± 88.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [864]:
%%timeit

np_weights.sum()

1.49 µs ± 8.16 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [910]:
@jit
def Numba_test(np_weights):
    sum = 0
    for i in np_weights:
        sum += i
    return sum

In [912]:
%%timeit
Numba_test(np_weights)

223 ns ± 8.29 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
%%cython -a

def QuickCalc_Exchange_Energy_Cython(crRNA,TargetSeq, new_weights):
    dG=0
    for i in range(len(crRNA)):
        pos=20-i
        if  crRNA[i] == TargetSeq[i]:
            continue
        else:
            dG+=new_weights[pos]
    return float(dG)


UsageError: Cell magic `%%cython` not found.


In [868]:
%%timeit
np_weights = np.array(new_weights)

QuickCalc_Exchange_Energy_Cython(guide, target_sequence, new_weights)

3.87 µs ± 66.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


__Converting the strings into numbers and getting Numba to work on a larger scale__

In [884]:
%%timeit
num_guide = np.array([ord(char) for char in list(guide)])

2.96 µs ± 79.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [863]:
%%timeit
num_guide = np.array([nt_pos[nt] for nt in list(guide)])

3.01 µs ± 326 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [892]:
%%timeit

np.array(genome_dictionary[source][fullPAM])

16 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [896]:
%%timeit

np_off_target_array = np.array([list[0] for list in genome_dictionary[source][fullPAM]])

3.34 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit
#Convert all off target sequences into a numpy number array
nt_pos={'A':0,'T':1,'C':2,'G':3}

for (source, targets) in genome_dictionary.items():
    np_dictionary = {}
    for fullPAM in returnAllPAMs():
        off_target_array = [sequence[0] for sequence in genome_dictionary[source][fullPAM]]
        Nucleotides = []
        for sequence in off_target_array:
            seq = []
            for nt in sequence:
                seq.append(nt_pos[nt])
            Nucleotides.append(np.array(seq))
        np_Nucleotides = np.array(Nucleotides)
        np_dictionary[fullPAM] = np_Nucleotides

np_weights = np.array([weight[0]*2.3 for weight in weights])

24.2 s ± 4.87 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit 
for (source, targets) in genome_dictionary.items():
    for fullPAM in returnAllPAMs():
        dG_PAM = 0
        dG_supercoiling= 0
        for off_target_sequence  in np_dictionary[fullPAM]:
            dG_exchange = QuickCalc_Exchange_Energy_Numba(num_guide, off_target_sequence, np_weights)

2.48 s ± 47.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
@jit(nopython = True, parallel = True)
def QuickCalc_Exchange_Energy_Numba_Parallel(crRNA,TargetSeq, np_weights):
    
    dG=0
    for i in prange(len(crRNA)):
        pos=20-i
        if  crRNA[i] == TargetSeq[i]:
            continue
        else:
            dG+=np_weights[pos]
    return float(dG)